## Bu çalışmada NCR (Uygunsuzluk Raporu) bilgileri ve NCR çeşidi verileri kullanılarak bir Doğal Dil İşleme (NLP) çalışması yapılmıştır.
### Daha fazla bilgi için https://www.linkedin.com/in/seyidcem-karaka%C5%9F-534848188/

---

## In this study, a Natural Language Processing (NLP) study was conducted using NCR (Nonconformity Report) information and NCR type data.
### For more information https://www.linkedin.com/in/seyidcem-karaka%C5%9F-534848188/

In [1]:
import pandas as pd
data=pd.read_excel("hata_tipleri.xlsx",engine="openpyxl")

Verimizi yükleyelim.(Bendeki office sürümü eski olduğu için **engine** parametresi işimi gördü.)

Let's upload our data. (Since the version of office I have is old, the ** engine ** parameter worked for me.)

In [2]:
data.head()

,hata,hata_tipi
0,Otopark temel süzgeç kenarı beton bitiş kusuru,İmalat Hatası
1,Otopark temel soğuk derzden beton dökülmemiş k...,İmalat Hatası
2,Otopark temeli üzeri mobil vinç ve beton pompa...,İmalat Hatası
3,Otopark temel soğuk derz kesim yerinden beton ...,İmalat Hatası
4,Betonarme imalat kalitesiyle ilgili uygulama s...,Yapım Yöntemine Uygunsuzluk


Verimizin ilk 5 satırına baktık.

We look at the first 5 lines of our data.

In [3]:
data["hata_tipi"].value_counts()

İmalat Hatası                  568
Yapım Yöntemine Uygunsuzluk    169
Malzeme Hatası                  63
Name: hata_tipi, dtype: int64

Hangi hata tipinden kaç adet var onu gördük.

We see how many of which error type there are.

---

Doğal dil işleme için **veri hazırlama/temizleme** sürecine başlayalım.

Let's start the **data preparation / cleaning** process for natural language processing.


In [4]:
import string
import re
import nltk
from nltk.corpus import stopwords

noktalama=string.punctuation
etkisiz=stopwords.words("turkish")

print(noktalama)
print(etkisiz)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani']


**nltk** paketinde türkçe işlemler için paket var.Noktalama işaretleri zaten çoğu dilde aynı (punctuation).

**nltk** package contains package for Turkish transactions. The punctuation marks are already the same in most languages (punctuation).

---

### Gereksiz değerlerin atılması (çalışma mantığı gösterimi)

### Discarding unnecessary values (working logic representation)

In [5]:
for d in data["hata"].head(5):
    print(d+"\n--------")
    temp=""
    for word in d.split():
        if word not in etkisiz and not word.isnumeric():
            temp+= word+" "
    print(temp+"\n****************")

Otopark temel süzgeç  kenarı beton bitiş kusuru
--------
Otopark temel süzgeç kenarı beton bitiş kusuru 
****************
Otopark temel soğuk derzden beton dökülmemiş kısıma çimento şerbeti sızması
--------
Otopark temel soğuk derzden beton dökülmemiş kısıma çimento şerbeti sızması 
****************
Otopark temeli üzeri mobil vinç ve beton pompasından zemine motor yağı sızması ve temel yüzeyine zarar verilmesi
--------
Otopark temeli üzeri mobil vinç beton pompasından zemine motor yağı sızması temel yüzeyine zarar verilmesi 
****************
Otopark temel soğuk derz kesim yerinden beton dökülmemiş bölüme beton ve çimento şerbeti sızması
--------
Otopark temel soğuk derz kesim yerinden beton dökülmemiş bölüme beton çimento şerbeti sızması 
****************
Betonarme imalat kalitesiyle ilgili uygulama sorunları
--------
Betonarme imalat kalitesiyle ilgili uygulama sorunları 
****************


### Noktalama işaretlerinin atılması (çalışma mantığı gösterimi)

### Discarding punctuation marks (working logic representation)

In [6]:
for d in data["hata"].tail(5):
    print(d+"\n------------")
    temp=""
    for word in d:
        if word not in noktalama:
            temp+= word
    print(temp+"\n*****************")
    d=temp

Zemin kat su basması hakkında
------------
Zemin kat su basması hakkında
*****************
VT14-15 asansör kuyu dibi döşemesine su akması, döşemedeki çelik profillerin ve baklava sacların paslanmaya başlaması 
------------
VT1415 asansör kuyu dibi döşemesine su akması döşemedeki çelik profillerin ve baklava sacların paslanmaya başlaması 
*****************
Çok amaçlı salon üzeri +6.00 kotu çelik döşemede INP300 kirişin iki ucunda civataların sıkılmaması ve sıkılmadan trapezin döşenmesi 
------------
Çok amaçlı salon üzeri 600 kotu çelik döşemede INP300 kirişin iki ucunda civataların sıkılmaması ve sıkılmadan trapezin döşenmesi 
*****************
Kule +209.00 kotu döşemesinde çatlak oluşması ve helikopter perdahı yapılmaması 
------------
Kule 20900 kotu döşemesinde çatlak oluşması ve helikopter perdahı yapılmaması 
*****************
Ayrık podyum 7.kat 3. etap (+35.80/+40.10 kotları arası) kolon ve perdeler, döşeme altı ve kat geneli kaba yapı kusurları
------------
Ayrık podyum 7kat 3 e

---

Şimdi her bir veri için hem noktalamadan hem de gereksiz sözcüklerden kurtulup yeni bir DataFrame yapalım.

Now let's get rid of both punctuation and unnecessary words for each data and make a new DataFrame.

In [7]:
cleaned_data=pd.DataFrame(columns=["hata"])
for d in data["hata"]:
    a=""
    temp=""
    for word in d.split():
        if word not in etkisiz and not word.isnumeric():
            temp+= word+" "
    for word in temp:
        if word not in noktalama:
            a+=word+""
    cleaned_data = cleaned_data.append({"hata":a},ignore_index=True)
cleaned_data=cleaned_data.join(data["hata_tipi"])

In [8]:
cleaned_data

,hata,hata_tipi
0,Otopark temel süzgeç kenarı beton bitiş kusuru,İmalat Hatası
1,Otopark temel soğuk derzden beton dökülmemiş k...,İmalat Hatası
2,Otopark temeli üzeri mobil vinç beton pompasın...,İmalat Hatası
3,Otopark temel soğuk derz kesim yerinden beton ...,İmalat Hatası
4,Betonarme imalat kalitesiyle ilgili uygulama s...,Yapım Yöntemine Uygunsuzluk
...,...,...
795,Zemin kat su basması hakkında,İmalat Hatası
796,VT1415 asansör kuyu dibi döşemesine su akması ...,İmalat Hatası
797,Çok amaçlı salon üzeri 600 kotu çelik döşemede...,İmalat Hatası
798,Kule 20900 kotu döşemesinde çatlak oluşması he...,İmalat Hatası


**cleaned_data** temizlenmiş veridir.

**cleaned_data** is the cleared data.

---
bu dataframe'i bir .csv uzantılı dosyaya kaydedip yeniden çekelim.

Let's save this dataframe to a .csv file and draw again.

In [9]:
cleaned_data.to_csv(r"./cleaned.csv",index=False)

In [16]:
cleaned_data=pd.read_csv("cleaned.csv")

In [17]:
cleaned_data

,hata,hata_tipi
0,Otopark temel süzgeç kenarı beton bitiş kusuru,İmalat Hatası
1,Otopark temel soğuk derzden beton dökülmemiş k...,İmalat Hatası
2,Otopark temeli üzeri mobil vinç beton pompasın...,İmalat Hatası
3,Otopark temel soğuk derz kesim yerinden beton ...,İmalat Hatası
4,Betonarme imalat kalitesiyle ilgili uygulama s...,Yapım Yöntemine Uygunsuzluk
...,...,...
795,Zemin kat su basması hakkında,İmalat Hatası
796,VT1415 asansör kuyu dibi döşemesine su akması ...,İmalat Hatası
797,Çok amaçlı salon üzeri 600 kotu çelik döşemede...,İmalat Hatası
798,Kule 20900 kotu döşemesinde çatlak oluşması he...,İmalat Hatası


#### Bu veri ön işlemden çıktı ve işleme hazır durumda.

#### This data has been preprocessed and ready for processing.
---

Verimizi eğitim ve test için ayrılarım.

Lets separate our data for training and testing.

In [98]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(cleaned_data["hata"],cleaned_data["hata_tipi"],test_size=0.3,random_state=2)
print(x_train.shape)
print(x_test.shape)

(560,)
(240,)


#### Eğitim ve test verisindeki dataların sayma vektörlerine bakalım.

#### Let's look at the counting vectors of the data in training and test data.

In [99]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect=CountVectorizer()
x_train_counts = count_vect.fit_transform(x_train)
x_test_counts = count_vect.transform(x_test)
print(x_train_counts.shape)
print(x_test_counts.shape)

(560, 1389)
(240, 1389)


640 adet cümle ve 1451 adet **token** var.

There are 640 sentences and 1451 ** tokens **.

---

## Şimdi de **TF*IDF** vektörünü oluşturalım.

### TF: kelimelerin 1 metinde kaç kere geçtiğini kodlar.

### IDF: 1 kelimenin kaç farklı metinde geçtiğini kodlar.

### TFIDF : 1 kelimenin döküman içindeki önemini gösterir.
---

## Now let's create the ** TF * IDF ** vector.

### TF: codes how many times words occur in 1 text.

### IDF: It encodes how many different text a word occurs.

### TFIDF: It shows the importance of 1 word in the document.

In [100]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)
x_test_tfidf = tfidf_transformer.transform(x_test_counts)
print(x_train_tfidf.shape)
print(x_test_tfidf.shape)

(560, 1389)
(240, 1389)


### Şimdi çok modlu Naive Bayes sınıflandırıcı model kuralım.

### Let's set up a multi-mode Naive Bayes classifier model.

In [101]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(x_train_tfidf,y_train)

### Tahmin yapalım.

### Let's guess.

In [102]:
y_pred = clf.predict(x_test_tfidf)

### Model performansına bakalım.

### Let's look at the model performance.

In [103]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.7583333333333333


Veri seti ve modele bağlı olarak değişen bu oran iyi düzeydedir.

This ratio, which varies depending on the data set and model, is at a good level.

---

random_state = 0 (veya herhangi bir sayı) yaptığımız için veri seti belli bir eyrden bölündü. Ama belki de eğitim için kötü bir yerden bölünde ve test kısmı bizi yanılttı. Bunun için random_state parametresini ortadan kaldırıp döngüye sokalım ve doğruluk oranını belirleyelim (K-fold gibi bir işlem).

Since we made random_state = 0 (or any number), the data set was split by a certain task. But maybe it's a bad place for education and the testing part has misled us. For this, let's remove the random_state parameter and loop it and determine its accuracy (a process like K-fold).

In [134]:
while True:
    x_train,x_test,y_train,y_test=train_test_split(cleaned_data["hata"],cleaned_data["hata_tipi"],test_size=0.3)
    count_vect=CountVectorizer()
    x_train_counts = count_vect.fit_transform(x_train)
    x_test_counts = count_vect.transform(x_test)
    tfidf_transformer = TfidfTransformer()
    x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)
    x_test_tfidf = tfidf_transformer.transform(x_test_counts)
    clf = MultinomialNB().fit(x_train_tfidf,y_train)
    y_pred = clf.predict(x_test_tfidf)
    if accuracy_score(y_test,y_pred) >0.81:
        print(f"!!!! 0.8 geçildi ===> {accuracy_score(y_test,y_pred)}")
        break

!!!! 0.8 geçildi ===> 0.825


In [135]:
dataset = pd.DataFrame({"Tahmin":y_pred,"Gerçek":y_test})
print(dataset.tail(30))
print(dataset.Tahmin.value_counts())
print(dataset.Gerçek.value_counts())

                          Tahmin                       Gerçek
578  Yapım Yöntemine Uygunsuzluk  Yapım Yöntemine Uygunsuzluk
278                İmalat Hatası                İmalat Hatası
711                İmalat Hatası                İmalat Hatası
25                 İmalat Hatası                İmalat Hatası
172                İmalat Hatası                İmalat Hatası
334                İmalat Hatası                İmalat Hatası
471                İmalat Hatası                İmalat Hatası
166                İmalat Hatası                İmalat Hatası
705                İmalat Hatası                İmalat Hatası
296                İmalat Hatası                İmalat Hatası
190                İmalat Hatası                İmalat Hatası
772                İmalat Hatası                İmalat Hatası
441                İmalat Hatası                İmalat Hatası
536                İmalat Hatası                İmalat Hatası
330                İmalat Hatası  Yapım Yöntemine Uygunsuzluk
302     